# Dataset Cleanup

This notebook demonstrates how the original 24 data files are combined into the 4 train and test sets provided.


# Notebook Preparation

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
threnjen_2019_airline_delays_and_cancellations_path = kagglehub.dataset_download('threnjen/2019-airline-delays-and-cancellations')

print('Data source import complete.')


100%|██████████| 754M/754M [00:04<00:00, 180MB/s]


Extracting files...
Data source import complete.


In [ ]:
threnjen_2019_airline_delays_and_cancellations_path

'/root/.cache/kagglehub/datasets/threnjen/2019-airline-delays-and-cancellations/versions/9'

In [ ]:
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/gdrive/')
from sklearn.model_selection import train_test_split

Mounted at /content/gdrive/


# Obtaining our Data

## About the Data

Our data comes from a variety of sources, all aimed at creating a full view of airport delay through various study metrics involving the airport, aircraft, airline, passengers, and weather.

Our primary dataset is the Bureau of Transportation Statistics' Montly On-Time Report, which for the year of 2019 comprises several million rows of data on every flight flown domestically for the entire year. We use and combine these monthly statistics with a variety of other data sets to gain further insights.

We use 5 informational datasets from the Bureau of Transportation Statistics:
* T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY.csv
* B43_AIRCRAFT_INVENTORY.csv
* AIRPORT_COORDINATES.csv
* CARRIER_DECODE.csv
* P10_EMPLOYEES.csv


2 informational datasets from the National Centers for Environmental Information
* Airport_Weather.csv
* Airport_list.csv

The data sets can be refined at download, so I chose features that I needed when acquiring the data.

Our base data of on-time reporting is feature rich. We have detailed information for EVERY flight taken, including the date, the carrier, the tail number, the origin airport, the destination airport, the time the flight left, the reason for delay if delayed, the length of the flight, and the distance it traveled on the flight. We are interested in the delay and will clean for both general delay and specific delay.

In [ ]:
# Load a month of data so we can see what kind of information we're working with
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path + '/raw_data/ONTIME_REPORTING_01.csv')
df.shape

(583985, 33)

In [ ]:
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
0,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,32.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
2,1,8,2,9E,N820AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,39.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,1,9,3,9E,N840AY,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,37.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10,4,9E,N8969A,3280,10397,ATL,"Atlanta, GA",11150,...,47.0,41.0,83.0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check memory usage of this file
df.memory_usage().sum()
# These files are LARGE, and this is only one month.
# Part of our cleaning process will be to store our data in a more memory efficient manner.

np.int64(154172172)

In [ ]:
# What types of data do we have? We can get a feel of what we may be able to reduce to a smaller integer
df.dtypes

,0
MONTH,int64
DAY_OF_MONTH,int64
DAY_OF_WEEK,int64
OP_UNIQUE_CARRIER,object
TAIL_NUM,object
OP_CARRIER_FL_NUM,int64
ORIGIN_AIRPORT_ID,int64
ORIGIN,object
ORIGIN_CITY_NAME,object
DEST_AIRPORT_ID,int64


In [ ]:
# Descriptive stats for our data
df.describe()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 32
count,583985.0,583985.000000,583985.000000,583985.000000,583985.000000,583985.000000,583985.000000,567633.000000,567630.000000,567630.000000,...,583851.000000,565963.000000,583985.000000,583985.000000,105222.000000,105222.000000,105222.000000,105222.000000,105222.000000,0.0
mean,1.0,15.960088,3.835626,2537.869334,12659.701982,12659.470015,1326.266642,1331.957814,13.258226,0.174281,...,143.557401,138.610077,797.742767,3.664861,20.106080,4.428551,18.508392,0.079508,24.950590,NaN
std,0.0,8.987942,1.921899,1821.736145,1519.405493,1519.336466,484.081000,495.404020,47.507390,0.379351,...,73.216303,73.078565,589.999261,2.323890,65.762202,38.535323,41.726392,3.203342,50.851973,NaN
min,1.0,1.000000,1.000000,1.000000,10135.000000,10135.000000,1.000000,1.000000,0.000000,0.000000,...,20.000000,16.000000,31.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,1.0,8.000000,2.000000,979.000000,11292.000000,11292.000000,917.000000,921.000000,0.000000,0.000000,...,90.000000,85.000000,363.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,1.0,16.000000,4.000000,2114.000000,12889.000000,12889.000000,1320.000000,1328.000000,0.000000,0.000000,...,125.000000,121.000000,640.000000,3.000000,0.000000,0.000000,5.000000,0.000000,0.000000,NaN
75%,1.0,24.000000,5.000000,3902.000000,13931.000000,13931.000000,1730.000000,1738.000000,5.000000,0.000000,...,175.000000,170.000000,1037.000000,5.000000,16.000000,0.000000,21.000000,0.000000,29.000000,NaN
max,1.0,31.000000,7.000000,7439.000000,16218.000000,16218.000000,2359.000000,2400.000000,1651.000000,1.000000,...,703.000000,737.000000,4983.000000,11.000000,1638.000000,1416.000000,1447.000000,816.000000,1514.000000,NaN


In [ ]:
# Check our missing data
df.isna().sum()

,0
MONTH,0
DAY_OF_MONTH,0
DAY_OF_WEEK,0
OP_UNIQUE_CARRIER,0
TAIL_NUM,2543
OP_CARRIER_FL_NUM,0
ORIGIN_AIRPORT_ID,0
ORIGIN,0
ORIGIN_CITY_NAME,0
DEST_AIRPORT_ID,0


# Scrubbing/Cleaning our Data

## Loading Data for Merging

T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY provides information on how many departures were performed (REV_ACRFT_DEP_PERF_510) and how many passengers were enplaned (REV_PAX_ENP_110) by CARRIER and AIRPORT. We'll use this data to provide metrics for the "busy-ness" of an airport and airline.

In [ ]:
# Information on passenger activity for airports and airlines
passengers = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path + '/raw_data/T3_AIR_CARRIER_SUMMARY_AIRPORT_ACTIVITY_2019.csv')
passengers

,OP_UNIQUE_CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,SERVICE_CLASS,REV_ACRFT_DEP_PERF_510,REV_PAX_ENP_110
0,04Q,Tradewind Aviation,15024,K,10.0,39.0
1,04Q,Tradewind Aviation,14843,K,677.0,3649.0
2,04Q,Tradewind Aviation,10257,V,4.0,6.0
3,04Q,Tradewind Aviation,15323,V,1.0,3.0
4,04Q,Tradewind Aviation,10158,V,1.0,2.0
...,...,...,...,...,...,...
27247,ZW,Air Wisconsin Airlines Corp,11637,K,122.0,4535.0
27248,ZW,Air Wisconsin Airlines Corp,11721,K,143.0,5800.0
27249,ZW,Air Wisconsin Airlines Corp,10469,K,248.0,8901.0
27250,ZW,Air Wisconsin Airlines Corp,12884,K,187.0,7923.0


B43_AIRCRAFT_INVENTORY provides information about specific tail numbers. We want to know the age of an aircraft, and how many passengers it seats.

In [ ]:
# Manufacture year and passenger capacity for aircraft by unique aircraft tail number
aircraft = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path + "/raw_data/B43_AIRCRAFT_INVENTORY.csv",encoding='latin1')
aircraft.drop_duplicates(subset='TAIL_NUM', inplace=True)
aircraft

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


AIRPORT_COORDINATES simply provides specific latitide/longitude for airports. We'll use this as location information.

In [ ]:
# coordinates of airports
coords = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path + '/raw_data/AIRPORT_COORDINATES.csv')
coords.drop_duplicates(subset='ORIGIN_AIRPORT_ID', inplace=True)
coords

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,LATITUDE,LONGITUDE
0,10001,Afognak Lake Airport,58.109444,-152.906667
1,10003,Bear Creek Mining Strip,65.548056,-161.071667
2,10004,Lik Mining Camp,68.083333,-163.166667
3,10005,Little Squaw Airport,67.570000,-148.183889
4,10006,Kizhuyak Bay,57.745278,-152.882778
...,...,...,...,...
18128,16908,Deer Park Airport,47.966944,-117.428611
18129,16909,South Texas International at Edinburg,26.441667,-98.122222
18130,16910,Louisa County Freeman Field,38.009722,-77.970000
18131,16911,Caldwell Industrial,43.641944,-116.635833


##### Testing to get DESTINATION AIRPORT

In [ ]:
# # DEST_AIRPORT_ID are airport_id's that are found in coords
# # Want to match airport_ids to get DISPLAY_AIRPORT_NAME for different destinations in df
# coords_indexed = coords.set_index('ORIGIN_AIRPORT_ID')
# df['DESTINATION_AIRPORT'] = df['DEST_AIRPORT_ID'].map(coords_indexed['DISPLAY_AIRPORT_NAME'])
# df

CARRIER_DECODE is to get a lookup table for airline codes to match into the main On-Time Reports.

In [ ]:
# proper names of carriers for better EDA usage
names = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+"/raw_data/CARRIER_DECODE.csv")
names.drop_duplicates(inplace=True)
names.drop_duplicates(subset=['OP_UNIQUE_CARRIER'], inplace=True)
names

,AIRLINE_ID,OP_UNIQUE_CARRIER,CARRIER_NAME
0,21754,2PQ,21 Air LLC
3,20342,Q5,40-Mile Air
4,20342,WRB,40-Mile Air
6,19627,CIQ,A/S Conair
7,19072,AAE,AAA Airlines
...,...,...,...
2702,20379,ZKQ,Zantop International
2706,19771,ZAQ,Zas Airline Of Egypt
2707,21118,37,Zeal 320
2708,22069,ZG,ZIPAIR Tokyo Inc.


P10_EMPLOYEES is so we can determine how many employees a carrier has for Passenger Handling (flight attendants) as well as Ground Service, so that we can determine the employees per passenger.

In [ ]:
# Employee statistics for carriers
employees = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path + '/raw_data/P10_EMPLOYEES.csv')
employees = employees[['OP_UNIQUE_CARRIER', 'PASS_GEN_SVC_ADMIN', 'PASSENGER_HANDLING']]
employees = employees.groupby('OP_UNIQUE_CARRIER').sum().reset_index()
employees

,OP_UNIQUE_CARRIER,PASS_GEN_SVC_ADMIN,PASSENGER_HANDLING
0,0WQ,19,0
1,1BQ,41,0
2,2HQ,24,0
3,3EQ,32,0
4,5V,0,0
5,5X,0,0
6,5Y,273,0
7,8C,37,0
8,9E,1361,0
9,9S,3,0


### Cleaning Weather Data

Weather Data was acquired on a daily basis for each airport used in the data set. We will ultimately use snowfall, ground snow, precipitation, wind speed, and temperature as features in our data set.

In [ ]:
# Weather report for top 90% of airport cities, in 2019
weather_report = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/airport_weather_2019.csv')
weather_report

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,...,WT08,WT09,WESD,WT10,PSUN,TSUN,SN32,SX32,TOBS,WT11
0,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/1/2019,4.70,NaN,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/2/2019,4.92,NaN,0.57,0.0,0.0,56.0,59.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/3/2019,5.37,NaN,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/4/2019,12.08,NaN,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,1/5/2019,13.42,NaN,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38670,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-27,6.04,NaN,0.00,NaN,NaN,68.0,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38671,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-28,5.37,NaN,0.06,NaN,NaN,69.0,74.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38672,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-29,7.61,NaN,0.10,NaN,NaN,70.0,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38673,USW00093805,"TALLAHASSEE REGIONAL AIRPORT, FL US",2019-12-30,5.82,NaN,0.02,NaN,NaN,68.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We also have a set of the airports and their city names to match up with the weather dataset, so that we can then match this up to our On-Time Dataset

In [ ]:
# Our list of cities and airports including the airport display name so that we can connect with our main df
cities = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/airports_list.csv')
cities

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
1,10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
2,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
3,10299,Anchorage International,"Anchorage, AK","ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, A..."
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
93,12264,Washington Dulles International,"Washington, DC","WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US"
94,13851,Will Rogers World,"Oklahoma City, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,12191,William P Hobby,"Houston, TX","HOUSTON WILLIAM P HOBBY AIRPORT, TX US"


In [ ]:
# Connect our weather report with the city names
weather_merge = pd.merge(cities, weather_report, how='left', on='NAME')
weather_merge.columns

Index(['ORIGIN_AIRPORT_ID', 'DISPLAY_AIRPORT_NAME', 'ORIGIN_CITY_NAME', 'NAME',
       'STATION', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG',
       'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03',
       'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WESD', 'WT10', 'PSUN',
       'TSUN', 'SN32', 'SX32', 'TOBS', 'WT11'],
      dtype='object')

In [ ]:
# Get just the important metrics from the weather report (date, precipitation, snow, max_temp, min_temp, wind, extreme_weather_codes WT...)
weather = weather_merge[['DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'AWND', 'ORIGIN_AIRPORT_ID', 'WT01', 'WT02', 'WT03',
       'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11']]

In [ ]:
weather

,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,ORIGIN_AIRPORT_ID,WT01,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11
0,2019-01-01,0.00,0.0,0.0,45.0,39.0,4.70,12992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-02,0.39,0.0,0.0,39.0,36.0,2.01,12992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-03,0.44,0.0,0.0,41.0,37.0,6.26,12992,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-04,0.13,0.0,0.0,47.0,37.0,2.01,12992,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-05,0.00,0.0,0.0,62.0,37.0,1.79,12992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35020,2019-12-27,0.00,0.0,0.0,35.0,26.0,5.82,10713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35021,2019-12-28,0.00,0.0,0.0,39.0,24.0,2.24,10713,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35022,2019-12-29,0.04,0.1,0.0,32.0,28.0,6.26,10713,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
35023,2019-12-30,0.00,0.0,0.0,34.0,29.0,2.46,10713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
weather_merge.notnull().sum()

,0
ORIGIN_AIRPORT_ID,35025
DISPLAY_AIRPORT_NAME,35025
ORIGIN_CITY_NAME,35025
NAME,35025
STATION,35024
DATE,35024
AWND,35016
PGTM,3434
PRCP,35003
SNOW,23965


In [ ]:
# Drop any rows where no weather was recorded
weather.drop(weather.loc[weather['ORIGIN_AIRPORT_ID'].isna()].index, axis=0, inplace=True)

In [ ]:
# Look for null values in temperature
weather['AWND_missing'] = (weather['AWND'].isna()).astype(int)
weather['TMIN_missing'] = (weather['TMIN'].isna()).astype(int)
weather['TMAX_missing'] = (weather['TMAX'].isna()).astype(int)
weather.loc[weather['TMAX'].isna()]

,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,ORIGIN_AIRPORT_ID,WT01,WT02,...,WT05,WT06,WT07,WT08,WT09,WT10,WT11,AWND_missing,TMIN_missing,TMAX_missing
4786,2/11/2019,0.22,NaN,0.0,NaN,NaN,9.62,11298,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1
19976,2019-10-06,0.00,NaN,0.0,NaN,NaN,12.30,15919,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1
24068,12/23/2019,0.00,NaN,NaN,NaN,29.0,1.57,11066,1.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0,0,1
24807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14843,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
30085,6/19/2019,NaN,NaN,0.0,NaN,NaN,7.61,14635,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1
31953,8/1/2019,0.66,0.0,0.0,NaN,73.0,7.61,15304,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1


In [ ]:
# Impute mean in nan rows for temp and wind
weather['TMAX'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather['TMIN'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['TMAX'].transform('mean'), 1), inplace=True)
weather['AWND'].fillna(round(weather.groupby('ORIGIN_AIRPORT_ID')['AWND'].transform('mean'), 1), inplace=True)
weather.fillna(0, inplace=True)

In [ ]:
# Check no NaN remain
weather.isna().sum()

,0
DATE,0
PRCP,0
SNOW,0
SNWD,0
TMAX,0
TMIN,0
AWND,0
ORIGIN_AIRPORT_ID,0
WT01,0
WT02,0


In [ ]:
# Cast data types to datetime so we can get the month and day of month to match up with main df
weather['DATE'] = pd.to_datetime(weather['DATE'], format='mixed')
weather['MONTH'] = pd.DatetimeIndex(weather['DATE']).month
weather['DAY_OF_MONTH'] = pd.DatetimeIndex(weather['DATE']).day
weather

,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,ORIGIN_AIRPORT_ID,WT01,WT02,...,WT07,WT08,WT09,WT10,WT11,AWND_missing,TMIN_missing,TMAX_missing,MONTH,DAY_OF_MONTH
0,2019-01-01,0.00,0.0,0.0,45.0,39.0,4.70,12992,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1
1,2019-01-02,0.39,0.0,0.0,39.0,36.0,2.01,12992,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,2
2,2019-01-03,0.44,0.0,0.0,41.0,37.0,6.26,12992,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,3
3,2019-01-04,0.13,0.0,0.0,47.0,37.0,2.01,12992,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,4
4,2019-01-05,0.00,0.0,0.0,62.0,37.0,1.79,12992,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35020,2019-12-27,0.00,0.0,0.0,35.0,26.0,5.82,10713,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,12,27
35021,2019-12-28,0.00,0.0,0.0,39.0,24.0,2.24,10713,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,12,28
35022,2019-12-29,0.04,0.1,0.0,32.0,28.0,6.26,10713,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0,0,0,12,29
35023,2019-12-30,0.00,0.0,0.0,34.0,29.0,2.46,10713,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,12,30


# Cleaning Function

## Editing to include DEP_DEL_NEW and OTHER FEATURE ENGINEERED VARIABLES

In [ ]:
def month_cleanup(monthly_data, aircraft, coords, names, weather, passengers, employees):

    '''Function which performs features engineering, data merges and cleanup using one month of On-Time data
    from Bureau of Transportation Services
    Parameters:
    monthly_data: month of on-time data as downloaded from BTS
    aircraft: Aircraft inventory data from BTS
    coords: Airport coordinates data from BTS
    names: Carrier names based on carrier code from BTS
    weather: Daily weather reported at airports from National Center for Environmental Information
    passengers: Yearly passenger information for carriers and airports from BTS
    employees: Employee statistics for carriers from BTS

    returns: cleaned month of On-Time reporting
    '''

    # start the timer so we can track how long the cleaning function takes
    start = time.time()

    # CLEANING
    # drop rows with no departure time, tail number, or were cancelled
    print("Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.")
    monthly_data.drop(monthly_data.loc[monthly_data['DEP_TIME'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['TAIL_NUM'].isna()].index, axis=0, inplace=True)
    monthly_data.drop(monthly_data.loc[monthly_data['CANCELLED']==1].index, axis=0, inplace=True)

    # FEATURE ENGINEERING - DEP HOUR
    # Extract departure hour from DEP_TIME, being mindful of 2400 maximum
    print("Adding Departure Hour - DEP_HOUR")
    monthly_data['DEP_HOUR'] = monthly_data['DEP_TIME']//100
    monthly_data['DEP_HOUR'] = np.where(monthly_data['DEP_HOUR'] == 24, 0, monthly_data['DEP_HOUR'])
    monthly_data['DEP_HOUR'].clip(0, 23) # clip purely for safety

    # FEATURE ENGINEERING - SEGMENT NUMBER
    # List flight segment number for daily flight segments by tracking tail number
    print("Adding Flight Number Sequence - SEGMENT_NUMBER")
    monthly_data["SEGMENT_NUMBER"] = monthly_data.groupby(["TAIL_NUM", 'DAY_OF_MONTH'])["DEP_TIME"].rank("dense", ascending=True)

    # FEATURE ENGINEERING - CONCURRENT FLIGHTS
    # Listing the number of concurrent flights at the airport in the time block
    print("Adding Concurrent Flights - CONCURRENT_FLIGHTS")
    monthly_data['CONCURRENT_FLIGHTS'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID','DAY_OF_MONTH', 'DEP_TIME_BLK'])['OP_UNIQUE_CARRIER'].transform("count")

    # MERGING to get NUMBER_OF_SEATS
    print("Applying seat counts to flights - NUMBER_OF_SEATS")
    # Merge aircraft info with main frame on tail number - get NUMBER_OF_SEATS
    monthly_data = pd.merge(monthly_data, aircraft, how="left", on='TAIL_NUM')
    # Fill missing aircraft info with means
    monthly_data['NUMBER_OF_SEATS'].fillna((monthly_data['NUMBER_OF_SEATS'].mean()), inplace=True)
    # simplify data type of number of seats to reduce memory usage
    monthly_data['NUMBER_OF_SEATS'] = monthly_data['NUMBER_OF_SEATS'].astype('int16')

    # MERGING
    # Merge to get proper carrier name
    print("Applying Carrier Names - CARRIER_NAME")
    monthly_data = pd.merge(monthly_data, names, how='left', on=['OP_UNIQUE_CARRIER'])

    # FEATURE ENGINEERING - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
    # Add monthly flight statistics for carrier and airport
    print("Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH")
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER'])['ORIGIN_CITY_NAME'].transform('count')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data.groupby(['OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID'])['ORIGIN_CITY_NAME'].transform('count')

    # FEATURE ENGINEERING - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
    #Add monthly passenger statistics for carrier and airport
    print("Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE")
    monthly_airport_passengers = pd.DataFrame(passengers.groupby(['ORIGIN_AIRPORT_ID'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airport_passengers, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['AVG_MONTHLY_PASS_AIRPORT'] = (monthly_data['REV_PAX_ENP_110']/12).astype('int64')
    monthly_airline_passengers = pd.DataFrame(passengers.groupby(['OP_UNIQUE_CARRIER'])['REV_PAX_ENP_110'].sum())
    monthly_data = pd.merge(monthly_data, monthly_airline_passengers, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['AVG_MONTHLY_PASS_AIRLINE'] = (monthly_data['REV_PAX_ENP_110_y']/12).astype('int64')

    # MERGING
    # Add employee stats then FEATURE ENGINEER FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
    print("Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS")
    monthly_data = pd.merge(monthly_data, employees, how='left', on=['OP_UNIQUE_CARRIER'])
    monthly_data['FLT_ATTENDANTS_PER_PASS'] = monthly_data['PASSENGER_HANDLING']/monthly_data['REV_PAX_ENP_110_y']
    monthly_data['GROUND_SERV_PER_PASS'] = monthly_data['PASS_GEN_SVC_ADMIN']/monthly_data['REV_PAX_ENP_110_y']

    # FEATURE ENGINEERING - PLANE AGE
    # Calculate age of plane
    print("Calculate Fleet Age - PLANE_AGE")
    monthly_data['MANUFACTURE_YEAR'].fillna((monthly_data['MANUFACTURE_YEAR'].mean()), inplace=True)
    monthly_data['PLANE_AGE'] = 2019 - monthly_data['MANUFACTURE_YEAR']

    # MERGING
    # Merge to get airport coordinates
    print("Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT")
    monthly_data = pd.merge(monthly_data, coords, how='left', on=['ORIGIN_AIRPORT_ID'])
    monthly_data['LATITUDE'] = round(monthly_data['LATITUDE'], 3)
    monthly_data['LONGITUDE'] = round(monthly_data['LONGITUDE'], 3)

    # FEATURE ENGINEERING - PREVIOUS AIRPORT
    # Get previous airport for tail number
    print("Adding airports - PREVIOUS_AIRPORT")
    segment_temp = monthly_data[['DAY_OF_MONTH', 'TAIL_NUM', 'DISPLAY_AIRPORT_NAME', 'SEGMENT_NUMBER']]
    monthly_data = pd.merge_asof(monthly_data.sort_values('SEGMENT_NUMBER'), segment_temp.sort_values('SEGMENT_NUMBER'), on='SEGMENT_NUMBER', by=['DAY_OF_MONTH', 'TAIL_NUM'], allow_exact_matches=False)
    monthly_data['DISPLAY_AIRPORT_NAME_y'].fillna('NONE', inplace=True)
    monthly_data.rename(columns={"DISPLAY_AIRPORT_NAME_y": "PREVIOUS_AIRPORT", "DISPLAY_AIRPORT_NAME_x": "DEPARTING_AIRPORT"}, inplace=True)

    # FEATURE ENGINEERING - DESTINATION AIRPORT
    # Get destination airport for tail number
    print("Adding airports - DESTINATION_AIRPORT")
    # Match DEST_AIRPORT_ID to ORIGIN_AIRPORT_ID in AIRPORT_COORDINATES
    # use that matching to get DISPLAY_AIRPORT_NAME for DEST_AIRPORT_ID
    coords_indexed = coords.set_index('ORIGIN_AIRPORT_ID')
    monthly_data['DESTINATION_AIRPORT'] = monthly_data['DEST_AIRPORT_ID'].map(coords_indexed['DISPLAY_AIRPORT_NAME'])
    monthly_data['DESTINATION_AIRPORT'].fillna('NONE', inplace=True)

    # FEATURE ENGINEERING - ROUTE_NAME
    ## Concatenate Departing and Destination
    print("Adding route name - ROUTE_NAME")
    monthly_data['ROUTE_NAME'] = monthly_data['DEPARTING_AIRPORT'] + ' - ' + monthly_data['DESTINATION_AIRPORT']

    # FEATURE ENGINEERING - INCOMING_ROUTE
    ## Concatenate Previous and Departing
    print("Adding incoming route - INCOMING_ROUTE")
    monthly_data['INCOMING_ROUTE'] = monthly_data['PREVIOUS_AIRPORT'] + ' - ' + monthly_data['DEPARTING_AIRPORT']

    # FEATURE ENGINEERING - CARRIER_AIRPORT
    ## Concatenate Carrier Name and Departing AIRPORT
    print("Adding Carrier + Airport (for potential Hub effects) - CARRIER_AIRPORT")
    monthly_data['CARRIER_AIRPORT'] = monthly_data['CARRIER_NAME'] + ' - ' + monthly_data['DEPARTING_AIRPORT']

    # CLEANING
    # Drop airports below the 10th percentile
    print("Dropping bottom 10% of airports")
    monthly_data.drop(monthly_data.loc[monthly_data['AIRPORT_FLIGHTS_MONTH'] < 1100].index, axis=0, inplace=True)

    # MERGING
    # Merge weather data
    print("Adding daily weather data - PRCP, SNOW, SNWD, TMAX, TMIN, AWND, and extreme weather encodings")
    monthly_data = pd.merge(monthly_data, weather, how='inner', on=['ORIGIN_AIRPORT_ID', 'MONTH', 'DAY_OF_MONTH'])

    # FEATURE ENGINEERING - TEMPERATURE AND WEATHER
    print("Adding weather severity thresholds")
    monthly_data['IS_HEAVY_RAIN'] = (monthly_data['PRCP'] > 0.05).astype(int)
    monthly_data['IS_SNOWY'] = (monthly_data['SNOW'] > 0.1).astype(int)
    monthly_data['IS_FREEZING'] = (monthly_data['TMIN'] < 32).astype(int)
    monthly_data['IS_EXTREME_HEAT'] = (monthly_data['TMAX'] > 95).astype(int)

    # FEATURE ENGINEERING - SEASONALITY
    # Winter = 0, Spring = 1, Summer = 2, Fall = 3 (CATEGORIES, not integers)
    season_map = {12:0, 1:0, 2:0, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3, 10:3, 11:3}
    monthly_data['SEASON'] = monthly_data['MONTH'].map(season_map).astype('category')

    # CLEANING
    # drop columns that we won't use
    print("Clean up unneeded columns")
    monthly_data.drop(columns = ['ORIGIN',  'DEST',
                   'CRS_DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME',
                   'CANCELLED', 'CANCELLATION_CODE', 'CRS_ELAPSED_TIME',
                  'ARR_DELAY_NEW', 'Unnamed: 32',  'ARR_TIME_BLK', 'ACTUAL_ELAPSED_TIME',
                  'DEST_AIRPORT_ID', 'DEST_CITY_NAME',  'OP_CARRIER_FL_NUM',  'OP_UNIQUE_CARRIER',
                       'AIRLINE_ID', 'DATE', 'DAY_OF_MONTH', 'TAIL_NUM',
                    'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME',  'PASSENGER_HANDLING', 'REV_PAX_ENP_110_x', 'REV_PAX_ENP_110_y',
                                 'PASS_GEN_SVC_ADMIN', 'MANUFACTURE_YEAR',
                                 ],
                    axis=1, inplace=True)

    # CLEANING
    # specify data types of various fields to reduce memory usage
    print("Cleaning up data types")
    monthly_data['MONTH'] = monthly_data['MONTH'].astype('object')
    monthly_data['DAY_OF_WEEK'] = monthly_data['DAY_OF_WEEK'].astype('object')
    monthly_data['DEP_DEL15'] = monthly_data['DEP_DEL15'].astype('int8')
    monthly_data['DEP_DELAY_NEW'] = monthly_data['DEP_DELAY_NEW'].astype('int8')
    monthly_data['DISTANCE'] = monthly_data['DISTANCE'].astype('int8')
    monthly_data['DISTANCE_GROUP'] = monthly_data['DISTANCE_GROUP'].astype('int8')
    monthly_data['SEGMENT_NUMBER'] = monthly_data['SEGMENT_NUMBER'].astype('int8')
    monthly_data['AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_FLIGHTS_MONTH'] = monthly_data['AIRLINE_FLIGHTS_MONTH'].astype('int64')
    monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'] = monthly_data['AIRLINE_AIRPORT_FLIGHTS_MONTH'].astype('int64')
    monthly_data['PLANE_AGE'] = monthly_data['PLANE_AGE'].astype('int32')

    # reset index
    monthly_data.reset_index(inplace=True, drop=True)

    # print elapsed time
    print(f'Elapsed Time: {time.time() - start}')

    print("FINISHED")

    # return cleaned file
    return monthly_data

# Apply Cleaning Function to All On-Time Reporting Datasets

This function processes our data set and combines it into one large master set.

In [ ]:
# Read and process each month of raw data using the cleaning function

df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_01.csv')
month01 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_02.csv')
month02 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_03.csv')
month03 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_04.csv')
month04 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_05.csv')
month05 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_06.csv')
month06 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_07.csv')
month07 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_08.csv')
month08 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_09.csv')
month09 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_10.csv')
month10 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_11.csv')
month11 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)
df = pd.read_csv(threnjen_2019_airline_delays_and_cancellations_path+'/raw_data/ONTIME_REPORTING_12.csv')
month12 = month_cleanup(df, aircraft, coords, names, weather, passengers, employees)


Dropping NaNs from Dep Time, Tail Num. Dropping Cancellations.
Adding Departure Hour - DEP_HOUR
Adding Flight Number Sequence - SEGMENT_NUMBER
Adding Concurrent Flights - CONCURRENT_FLIGHTS
Applying seat counts to flights - NUMBER_OF_SEATS
Applying Carrier Names - CARRIER_NAME
Adding flight statistics for carrier and airport - AIRPORT_FLIGHTS_MONTH, AIRLINE_FLIGHTS_MONTH, AIRLINE_AIRPORT_FLIGHTS_MONTH
Adding passenger statistics for carrier and airport - AVG_MONTHLY_PASS_AIRPORT, AVG_MONTHLY_PASS_AIRLINE
Adding employee statistics for carrier - FLT_ATTENDANTS_PER_PASS, GROUND_SERV_PER_PASS
Calculate Fleet Age - PLANE_AGE
Adding airport coordinates - LATITUDE, LONGITUDE, DEPARTING_AIRPORT
Adding airports - PREVIOUS_AIRPORT
Adding airports - DESTINATION_AIRPORT
Adding route name - ROUTE_NAME
Adding incoming route - INCOMING_ROUTE
Adding Carrier + Airport (for potential Hub effects) - CARRIER_AIRPORT
Dropping bottom 10% of airports
Adding daily weather data - PRCP, SNOW, SNWD, TMAX, TMIN,

In [ ]:
all_data = pd.concat([month01, month02, month03, month04, month05, month06, month07, month08, month09, month10, month11, month12]).reset_index(drop=True)

In [ ]:
all_data.columns

Index(['MONTH', 'DAY_OF_WEEK', 'DEP_TIME', 'DEP_DELAY_NEW', 'DEP_DEL15',
       'DEP_TIME_BLK', 'DISTANCE', 'DISTANCE_GROUP', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'DEP_HOUR', 'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS',
       'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT',
       'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS',
       'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'DEPARTING_AIRPORT', 'LATITUDE',
       'LONGITUDE', 'PREVIOUS_AIRPORT', 'DESTINATION_AIRPORT', 'ROUTE_NAME',
       'INCOMING_ROUTE', 'CARRIER_AIRPORT', 'PRCP', 'SNOW', 'SNWD', 'TMAX',
       'TMIN', 'AWND', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07',
       'WT08', 'WT09', 'WT10', 'WT11', 'AWND_missing', 'TMIN_missing',
       'TMAX_missing', 'IS_HEAVY_RAIN', 'IS_SNOWY', 'IS_FREEZING',
       'IS_EXTREME_HEAT', 'SEASON'],
      dtype='o

In [ ]:
all_data.shape

(6489062, 59)

In [ ]:
all_data['ROUTE_NAME'].value_counts(normalize=True).head(20)

,proportion
ROUTE_NAME,
Chicago O'Hare International - LaGuardia,0.002191
LaGuardia - Chicago O'Hare International,0.002189
Los Angeles International - San Francisco International,0.002172
San Francisco International - Los Angeles International,0.002171
Los Angeles International - John F. Kennedy International,0.001972
John F. Kennedy International - Los Angeles International,0.001963
Los Angeles International - McCarran International,0.001792
McCarran International - Los Angeles International,0.001790
Honolulu International - Kahului Airport,0.001653


## Drop negative delays and "mislabeled" data
mislabeled data == data that has DEP_DEL15 = 1 but DEP_DELAY_NEW < 15 (there aren't any of the opposite, where DEP_DEL15 == 0 and DEP_DELAY_NEW >= 15)



In [ ]:
# df.loc[df['DEP_DELAY_NEW'] < 0].index

In [ ]:
all_data.describe()

,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,...,WT10,WT11,AWND_missing,TMIN_missing,TMAX_missing,IS_HEAVY_RAIN,IS_SNOWY,IS_FREEZING,IS_EXTREME_HEAT,SEASON
count,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,1.246999e+06,1.246999e+06,1.246999e+06,1.246999e+06,1.246999e+06,...,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06,6.489062e+06
mean,1.350487e+03,7.111874e+00,1.891441e-01,-2.767354e+00,3.821102e+00,2.063721e+01,3.514959e+00,1.649660e+01,9.512277e-02,2.665774e+01,...,1.191235e-04,1.587286e-05,1.148086e-04,2.476475e-04,1.998748e-04,2.078650e-01,2.069883e-02,1.273341e-01,5.227135e-02,1.532520e+00
std,5.084784e+02,2.454840e+01,3.916231e-01,7.472465e+01,2.382233e+00,6.221202e+01,3.032180e+01,3.880711e+01,3.427895e+00,5.182926e+01,...,1.091372e-02,3.984045e-03,1.071426e-02,1.573487e-02,1.413630e-02,4.057797e-01,1.423741e-01,3.333469e-01,2.225737e-01,1.101442e+00
min,1.000000e+00,-1.280000e+02,0.000000e+00,-1.280000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.240000e+02,0.000000e+00,0.000000e+00,-7.000000e+01,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,1.337000e+03,0.000000e+00,0.000000e+00,-6.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
75%,1.757000e+03,5.000000e+00,0.000000e+00,6.700000e+01,5.000000e+00,1.800000e+01,0.000000e+00,2.000000e+01,0.000000e+00,3.300000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
max,2.400000e+03,1.270000e+02,1.000000e+00,1.270000e+02,1.100000e+01,2.695000e+03,1.575000e+03,1.711000e+03,1.078000e+03,2.010000e+03,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00


In [ ]:
# drop negative delays; somehow early departures?
all_data_preprocessed = all_data.drop(all_data.loc[all_data['DEP_DELAY_NEW'] < 0].index, axis=0)

In [ ]:
all_data_preprocessed['delay_mismatch'] = (all_data_preprocessed['DEP_DELAY_NEW'] >= 15) != (all_data_preprocessed['DEP_DEL15'])
print(all_data_preprocessed['delay_mismatch'].value_counts())

delay_mismatch
False    6357979
True        5222
Name: count, dtype: int64


In [ ]:
all_data_preprocessed.loc[all_data_preprocessed['delay_mismatch'] == True].head(5)

,MONTH,DAY_OF_WEEK,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,...,WT11,AWND_missing,TMIN_missing,TMAX_missing,IS_HEAVY_RAIN,IS_SNOWY,IS_FREEZING,IS_EXTREME_HEAT,SEASON,delay_mismatch
3556,1,1,956.0,9,1,2000-2059,-69,1,0.0,775.0,...,0.0,0,0,0,0,0,1,0,0,True
5202,1,4,1428.0,12,1,1000-1059,-106,10,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,True
9154,1,5,1116.0,0,1,0700-0759,-81,2,0.0,0.0,...,0.0,0,0,0,0,0,0,0,0,True
10028,1,4,9.0,8,1,1900-1959,32,2,0.0,0.0,...,0.0,0,0,0,1,1,1,0,0,True
13196,1,1,1504.0,13,1,1000-1059,-64,8,253.0,0.0,...,0.0,0,0,0,0,0,0,0,0,True


In [ ]:
# get rows where delay_mismatch is True but dep_del15 is 0
all_data_preprocessed.loc[(all_data_preprocessed['delay_mismatch'] == True) & (all_data_preprocessed['DEP_DEL15'] ==0)].head(5)

,MONTH,DAY_OF_WEEK,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,...,WT11,AWND_missing,TMIN_missing,TMAX_missing,IS_HEAVY_RAIN,IS_SNOWY,IS_FREEZING,IS_EXTREME_HEAT,SEASON,delay_mismatch


In [ ]:
all_data_preprocessed.drop(all_data_preprocessed.loc[all_data_preprocessed['delay_mismatch'] == True].index, axis=0, inplace=True)

In [ ]:
all_data_preprocessed['delay_mismatch'].value_counts()

,count
delay_mismatch,
False,6357979


In [ ]:
all_data_preprocessed.drop(columns=['delay_mismatch'], inplace=True)

## Drop unwanted columns: distance, dep_del15

In [ ]:
## should probably drop the delay flags as well...?
all_data_preprocessed.drop(columns=['DISTANCE', 'LATITUDE', 'LONGITUDE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], inplace=True)

In [ ]:
all_data_preprocessed.columns

Index(['MONTH', 'DAY_OF_WEEK', 'DEP_TIME', 'DEP_DELAY_NEW', 'DEP_DEL15',
       'DEP_TIME_BLK', 'DISTANCE_GROUP', 'DEP_HOUR', 'SEGMENT_NUMBER',
       'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS', 'CARRIER_NAME',
       'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT',
       'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS',
       'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'DEPARTING_AIRPORT',
       'PREVIOUS_AIRPORT', 'DESTINATION_AIRPORT', 'ROUTE_NAME',
       'INCOMING_ROUTE', 'CARRIER_AIRPORT', 'PRCP', 'SNOW', 'SNWD', 'TMAX',
       'TMIN', 'AWND', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07',
       'WT08', 'WT09', 'WT10', 'WT11', 'AWND_missing', 'TMIN_missing',
       'TMAX_missing', 'IS_HEAVY_RAIN', 'IS_SNOWY', 'IS_FREEZING',
       'IS_EXTREME_HEAT', 'SEASON'],
      dtype='object')

## Feature Engineer DEP_ADDED_DELAY, the added delay beyond 14 minutes and clipped to 0 from the left to ensure non-negativity

In [ ]:
all_data_preprocessed['DEP_ADDED_DELAY'] = (all_data_preprocessed['DEP_DELAY_NEW'] - 14).clip(lower=0, upper=None)


In [ ]:
print(all_data_preprocessed['DEP_ADDED_DELAY'].value_counts(normalize=True)) # should be same value counts as DEP_DEL15, at least for 0 and sum(1-inf)
all_data_preprocessed['DEP_ADDED_DELAY'].value_counts(normalize=True).values[1:].sum()

DEP_ADDED_DELAY
0      0.827573
1      0.006406
2      0.005920
3      0.005601
4      0.005297
         ...   
107    0.000363
111    0.000360
110    0.000355
113    0.000348
112    0.000332
Name: proportion, Length: 114, dtype: float64


np.float64(0.17242664689518478)

In [ ]:
all_data_preprocessed['DEP_DEL15'].value_counts(normalize=True)

,proportion
DEP_DEL15,
0,0.827573
1,0.172427


## Save Data

In [ ]:
all_data_preprocessed.head(5)


,MONTH,DAY_OF_WEEK,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,DEP_HOUR,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,...,WT11,AWND_missing,TMIN_missing,TMAX_missing,IS_HEAVY_RAIN,IS_SNOWY,IS_FREEZING,IS_EXTREME_HEAT,SEASON,DEP_ADDED_DELAY
0,1,3,1652.0,0,0,1600-1659,7,16.0,1,25,...,0.0,0,0,0,1,1,1,0,0,0
1,1,3,822.0,0,0,0800-0859,10,8.0,1,34,...,0.0,0,0,0,0,0,1,0,0,0
2,1,3,644.0,44,1,0600-0659,4,6.0,1,16,...,0.0,0,0,0,0,0,1,0,0,30
4,1,3,556.0,0,0,0600-0659,3,5.0,1,11,...,0.0,0,0,0,0,0,1,0,0,0
5,1,3,607.0,0,0,0600-0659,2,6.0,1,30,...,0.0,0,0,0,0,0,0,0,0,0


In [ ]:
import datetime

file_path = '/content/gdrive/My Drive/CIS 5200 Final Project/merged_dataset/'
# COMBINE MASTER FILE
# safe saving with timestamps
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
all_data_preprocessed.to_pickle(file_path+f"full_dataset_added_delay_{time}.pkl")
all_data_preprocessed.to_csv(file_path+f'full_dataset_added_delay_{time}.csv', index=False)

# Train and Test Split

## Split the Full Dataset to 1 Train-Test Split (now that we have more ram)

In [ ]:
CLASS_TARGET = 'DEP_DEL15'
# Shifted and Clipped version of DEP_DELAY_NEW (- 14, clipped from left to 0 to enforce non-negativity)
NEW_REGR_TARGET = 'DEP_ADDED_DELAY'
OLD_REGR_TARGET = 'DEP_DELAY_NEW'

In [ ]:
# Ensure delay_flag column is present before splitting if it was temporary for sampling
if CLASS_TARGET not in df.columns:
    raise ValueError('Classification target for stratifying train_test data is not present')

# Define features (X) and target (y)
# X will contain all columns except 'DEP_DELAY_NEW' and delay_flag
# y will be the delay_flag column for stratification and binary classification

X = all_data_preprocessed.drop(columns=[CLASS_TARGET, NEW_REGR_TARGET, OLD_REGR_TARGET])
y = all_data_preprocessed[[CLASS_TARGET, NEW_REGR_TARGET, OLD_REGR_TARGET]]

In [ ]:
X.head(5)

,MONTH,DAY_OF_WEEK,DEP_TIME,DEP_TIME_BLK,DISTANCE_GROUP,DEP_HOUR,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,...,WT10,WT11,AWND_missing,TMIN_missing,TMAX_missing,IS_HEAVY_RAIN,IS_SNOWY,IS_FREEZING,IS_EXTREME_HEAT,SEASON
0,1,3,1652.0,1600-1659,7,16.0,1,25,154,United Air Lines Inc.,...,0.0,0.0,0,0,0,1,1,1,0,0
1,1,3,822.0,0800-0859,10,8.0,1,34,154,United Air Lines Inc.,...,0.0,0.0,0,0,0,0,0,1,0,0
2,1,3,644.0,0600-0659,4,6.0,1,16,154,United Air Lines Inc.,...,0.0,0.0,0,0,0,0,0,1,0,0
4,1,3,556.0,0600-0659,3,5.0,1,11,180,Delta Air Lines Inc.,...,0.0,0.0,0,0,0,0,0,1,0,0
5,1,3,607.0,0600-0659,2,6.0,1,30,110,Delta Air Lines Inc.,...,0.0,0.0,0,0,0,0,0,0,0,0


In [ ]:
y.head(5)

,DEP_DEL15,DEP_ADDED_DELAY,DEP_DELAY_NEW
0,0,0,0
1,0,0,0
2,1,30,44
4,0,0,0
5,0,0,0


In [ ]:
import datetime
TEST_RATIO = 0.2 # 80:20 split
 # Perform 80:20 train test split on full dataset
X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=TEST_RATIO,
        stratify=y[CLASS_TARGET],
        random_state=343 #
        )
print(f"Full Dataset: Train size={len(X_train)} ({y_train[CLASS_TARGET].mean():.4f} delayed ratio); "
      f"Test size={len(X_test)} ({y_test[CLASS_TARGET].mean():.4f} delayed ratio)")
print("-" * 50)

time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# Store the results
save_path = '/content/gdrive/My Drive/CIS 5200 Final Project/train_test_datasets'
X_train.to_csv(f'{save_path}/X_train_added_delay_{time}.csv', index=False)
X_test.to_csv(f'{save_path}/X_test_added_delay_{time}.csv', index=False)

y_train.to_csv(f'{save_path}/y_train_added_delay_{time}.csv', index=False)
y_test.to_csv(f'{save_path}/y_test_added_delay_{time}.csv', index=False)

Full Dataset: Train size=5086383 (0.1724 delayed ratio); Test size=1271596 (0.1724 delayed ratio)
--------------------------------------------------


In [ ]:
5086383 + 1271596

6357979

In [ ]:
X.columns

Index(['MONTH', 'DAY_OF_WEEK', 'DEP_TIME', 'DEP_TIME_BLK', 'DISTANCE_GROUP',
       'DEP_HOUR', 'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'NUMBER_OF_SEATS',
       'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'AVG_MONTHLY_PASS_AIRPORT',
       'AVG_MONTHLY_PASS_AIRLINE', 'FLT_ATTENDANTS_PER_PASS',
       'GROUND_SERV_PER_PASS', 'PLANE_AGE', 'DEPARTING_AIRPORT',
       'PREVIOUS_AIRPORT', 'DESTINATION_AIRPORT', 'ROUTE_NAME',
       'INCOMING_ROUTE', 'CARRIER_AIRPORT', 'PRCP', 'SNOW', 'SNWD', 'TMAX',
       'TMIN', 'AWND', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07',
       'WT08', 'WT09', 'WT10', 'WT11', 'AWND_missing', 'TMIN_missing',
       'TMAX_missing', 'IS_HEAVY_RAIN', 'IS_SNOWY', 'IS_FREEZING',
       'IS_EXTREME_HEAT', 'SEASON'],
      dtype='object')

## Subsample and Split 10 Datasets

In [ ]:
# from typing import List, Tuple, Dict

# # --- FUNCTION: Stratified Train/Test Split ---

# def split_subsample(X_sub: pd.DataFrame,
#                     y_sub: pd.DataFrame,
#                     test_ratio: float = 0.2,
#                     stratify_column: str = 'DEP_DEL15',
#                     random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
#     """
#     Splits a single subsample into stratified train and test sets.
#     """
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_sub,
#         y_sub,
#         test_size=test_ratio,
#         stratify=y_sub[stratify_column],
#         random_state=random_state
#     )
#     return X_train, X_test, y_train, y_test

In [ ]:
# # ====================================================================
# # --- 4. EXECUTION ---
# # ====================================================================

# N_SUBSAMPLES = 10
# SUBSAMPLE_SIZE = 1500000
# TEST_RATIO = 0.2 # 80:20 split

# print("\n--- 2. Creating and Splitting Each Subsample (80:20 Stratified) ---")
# for i in trange(N_SUBSAMPLES):
#     # We use train_test_split logic with test_size=subsample_size/N_TOTAL
#     # to select the desired subset while stratifying. The 'train' set is discarded.
#     # Note: Set random_state to None for true randomness in each iteration.

#     _, X_sub, _, y_sub = train_test_split(
#         X,
#         y,
#         test_size=SUBSAMPLE_SIZE,  # Directly use the desired size
#         stratify=y[delay_flag],
#         random_state=i # Use a different seed for each sample
#     )
#     print(f"Generated Subsample {i+1}/{N_SUBSAMPLES}. Delayed Ratio: {y_sub[delay_flag].mean():.4f}")

#     # Perform the stratified 80:20 split on the current subsample
#     X_train, X_test, y_train, y_test = split_subsample(
#         X_sub=X_sub,
#         y_sub=y_sub,
#         test_ratio=TEST_RATIO,
#         random_state=i # Use a different seed for each split
#     )
#     print(f"Sample {i+1}: Train size={len(X_train)} ({y_train[delay_flag].mean():.4f} delayed ratio); "
#           f"Test size={len(X_test)} ({y_test[delay_flag].mean():.4f} delayed ratio)")
#     print("-" * 50)

#     # Store the results
#     X_train.to_csv(f'/content/gdrive/My Drive/CIS 5200 Final Project/train_test_datasets/no_encodings/X_train_{i}_no_encoding.csv', index=False)
#     X_test.to_csv(f'/content/gdrive/My Drive/CIS 5200 Final Project/train_test_datasets/no_encodings/X_test_{i}_no_encoding.csv', index=False)

#     y_train.to_csv(f'/content/gdrive/My Drive/CIS 5200 Final Project/train_test_datasets/no_encodings/y_train_{i}_no_encoding.csv', index=False)
#     y_test.to_csv(f'/content/gdrive/My Drive/CIS 5200 Final Project/train_test_datasets/no_encodings/y_test_{i}_no_encoding.csv', index=False)

# print("\n--- Process Complete ---")
# print(f"You now have 10 independent dictionaries in 'all_datasets', each containing X_train, X_test, y_train, and y_test.")


--- 2. Creating and Splitting Each Subsample (80:20 Stratified) ---


  0%|          | 0/10 [00:00<?, ?it/s]

Generated Subsample 1/10. Delayed Ratio: 0.1724
Sample 1: Train size=1200000 (0.1724 delayed ratio); Test size=300000 (0.1724 delayed ratio)
--------------------------------------------------


 10%|█         | 1/10 [00:39<05:59, 39.96s/it]

Generated Subsample 2/10. Delayed Ratio: 0.1724
Sample 2: Train size=1200000 (0.1724 delayed ratio); Test size=300000 (0.1724 delayed ratio)
--------------------------------------------------


...

Generated Subsample 9/10. Delayed Ratio: 0.1724
Sample 9: Train size=1200000 (0.1724 delayed ratio); Test size=300000 (0.1724 delayed ratio)
--------------------------------------------------


 90%|█████████ | 9/10 [05:37<00:37, 37.61s/it]

Generated Subsample 10/10. Delayed Ratio: 0.1724
Sample 10: Train size=1200000 (0.1724 delayed ratio); Test size=300000 (0.1724 delayed ratio)
--------------------------------------------------


100%|██████████| 10/10 [06:14<00:00, 37.48s/it]


--- Process Complete ---
You now have 10 independent dictionaries in 'all_datasets', each containing X_train, X_test, y_train, and y_test.


# DONE - Split and Encodings done elsewhere
This code is from the original notebook from Jen Wadkins on kaggle

## Data Split - Test and Validation

We need to split our data into a train and testing set, because we will be using target encoding on our data. It's important that our target encoding not utilize any information found in either our Validation or our Test sets of new, unseen data. In fact, we use the target encoding that is created in our Train set to populate our Val and Test set target encodings, ensuring that our Validation/Test sets contain no leakage.

In [ ]:
# # Split into subsets
# train, test = train_test_split(all_data, test_size=0.2, random_state=42, stratify=all_data['DEP_DEL15'])

# # Our Train set
# train.head()

In [ ]:
# # Our Validation set
# test.head()

## Target Encoding

For our target encoding, we will group monthly delay statistics by the following categories:

- Carrier
- Airport (Use for both departing airport and arriving airport)
- Day of Week
- Departure Block

In [ ]:
# # Create lookup tables

# carrier_historical = pd.DataFrame(train.groupby(['CARRIER_NAME', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
# carrier_historical.rename(columns={'DEP_DEL15':'CARRIER_HISTORICAL'}, inplace=True)

# airport_historical = pd.DataFrame(train.groupby(['DEPARTING_AIRPORT', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
# airport_historical.rename(columns={'DEP_DEL15':'DEP_AIRPORT_HIST'}, inplace=True)

# prev_airport_historical = airport_historical
# prev_airport_historical.rename(columns={'DEPARTING_AIRPORT':'PREVIOUS_AIRPORT', 'DEP_DEL15':'PREV_AIRPORT_HIST'}, inplace=True)

# day_historical = pd.DataFrame(train.groupby(['DAY_OF_WEEK', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
# day_historical.rename(columns={'DEP_DEL15':'DAY_HISTORICAL'}, inplace=True)

# dep_block_lookup = pd.DataFrame(train.groupby(['DEP_TIME_BLK', 'MONTH'])['DEP_DEL15'].mean().transpose().reset_index())
# dep_block_lookup.rename(columns={'DEP_DEL15':'DEP_BLOCK_HIST'}, inplace=True)

In [ ]:
# # Merge lookup tables back onto data frame

# train = pd.merge(train, carrier_historical, how='left')
# train = pd.merge(train, airport_historical, how='left')
# train = pd.merge(train, prev_airport_historical, how='left')
# train = pd.merge(train, day_historical, how='left')
# train = pd.merge(train, dep_block_lookup, how='left')

In [ ]:
# # Now merge these lookup tables onto our Validation data frame

# test = pd.merge(test, carrier_historical, how='left')
# test = pd.merge(test, airport_historical, how='left')
# test = pd.merge(test, prev_airport_historical, how='left')
# test = pd.merge(test, day_historical, how='left')
# test = pd.merge(test, dep_block_lookup, how='left')

In [ ]:
# # Fillna with mean
# train['DEP_AIRPORT_HIST'].fillna(train['DEP_AIRPORT_HIST'].mean(), inplace=True)

# # save files to pickle
# train.to_pickle("data/pkl/train.pkl")
# train.to_csv('data/train.csv', index=False)

In [ ]:
# # fillna with mean
# test['DEP_AIRPORT_HIST'].fillna(test['DEP_AIRPORT_HIST'].mean(), inplace=True)

# # save to pickle
# test.to_pickle("data/pkl/test.pkl")
# test.to_csv('data/test.csv', index=False)